# Split the data into training and test sets

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
sample_metadata = pd.read_csv("../data/GSE131512_metaData.tsv", sep="\t", index_col=0)
sample_metadata

,Age,Gender
Sample name,,
C1,34.44,Female
C2,43.35,Female
C3,43.85,Female
C4,41.42,Female
C5,42.12,Female
...,...,...
N28,29.00,Female
N29,32.00,Female
N30,41.00,Female


In [15]:
cancer_samples = [i for i in sample_metadata.index if i[0] == "C"]
noncancer_samples = [i for i in sample_metadata.index if i[0] == "N"]
train_test_split(cancer_samples)

[['C93',
  'C47',
  'C80',
  'C68',
  'C40',
  'C31',
  'C22',
  'C39',
  'C67',
  'C85',
  'C3',
  'C74',
  'C41',
  'C95',
  'C13',
  'C63',
  'C5',
  'C11',
  'C66',
  'C78',
  'C8',
  'C50',
  'C29',
  'C42',
  'C64',
  'C45',
  'C56',
  'C69',
  'C33',
  'C86',
  'C72',
  'C81',
  'C70',
  'C27',
  'C62',
  'C21',
  'C17',
  'C19',
  'C48',
  'C88',
  'C44',
  'C35',
  'C52',
  'C10',
  'C14',
  'C84',
  'C38',
  'C60',
  'C46',
  'C37',
  'C83',
  'C51',
  'C65',
  'C82',
  'C61',
  'C91',
  'C2',
  'C18',
  'C90',
  'C16',
  'C9',
  'C89',
  'C28',
  'C79',
  'C4',
  'C15',
  'C55',
  'C96',
  'C77',
  'C75',
  'C34',
  'C43'],
 ['C59',
  'C26',
  'C7',
  'C25',
  'C23',
  'C71',
  'C49',
  'C58',
  'C73',
  'C12',
  'C57',
  'C20',
  'C87',
  'C53',
  'C36',
  'C30',
  'C94',
  'C24',
  'C6',
  'C1',
  'C54',
  'C76',
  'C32',
  'C92']]